In [1]:
import findspark
findspark.init()
import pyspark

- Importing and initiate findspark to begin with. And import Pyspark
- Now Start SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

- Create DataFrame from data source -csv file

In [3]:
customerDF = spark.read.load("sample_comma.txt" ,format="csv" , sep="," ,inferSchema="true", header="true")

In [4]:
customerDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zipcode: integer (nullable = true)



In [4]:
customerDF.collect()

[Row(id=321, name='Anubhaw', state='Jharkhand', city='Jamshedpur', zipcode=831015),
 Row(id=432, name='Ramesh', state='Bihar', city='Patna', zipcode=243244),
 Row(id=132, name='Manish', state='Karnataka', city='Jamshedpur', zipcode=254355),
 Row(id=632, name='Ankit', state='Panjab', city='Amritsar', zipcode=642345),
 Row(id=126, name='Aman', state='Maharastra', city='Pune', zipcode=345255),
 Row(id=732, name='Ravi', state='Rajasthan', city='kota', zipcode=246262)]

In [5]:
customerDF.show()

+---+-------+----------+----------+-------+
| id|   name|     state|      city|zipcode|
+---+-------+----------+----------+-------+
|321|Anubhaw| Jharkhand|Jamshedpur| 831015|
|432| Ramesh|     Bihar|     Patna| 243244|
|132| Manish| Karnataka|Jamshedpur| 254355|
|632|  Ankit|    Panjab|  Amritsar| 642345|
|126|   Aman|Maharastra|      Pune| 345255|
|732|   Ravi| Rajasthan|      kota| 246262|
+---+-------+----------+----------+-------+



### Different syntax for fetching the data

In [5]:
customerDF.select("name").show()

+-------+
|   name|
+-------+
|Anubhaw|
| Ramesh|
| Manish|
|  Ankit|
|   Aman|
|   Ravi|
+-------+



In [6]:
customerDF.select(["id","name","zipcode","city","state"]).show()

+---+-------+-------+----------+----------+
| id|   name|zipcode|      city|     state|
+---+-------+-------+----------+----------+
|321|Anubhaw| 831015|Jamshedpur| Jharkhand|
|432| Ramesh| 243244|     Patna|     Bihar|
|132| Manish| 254355|Jamshedpur| Karnataka|
|632|  Ankit| 642345|  Amritsar|    Panjab|
|126|   Aman| 345255|      Pune|Maharastra|
|732|   Ravi| 246262|      kota| Rajasthan|
+---+-------+-------+----------+----------+



In [7]:
customerDF.select(customerDF['id'],customerDF['name']).show()

+---+-------+
| id|   name|
+---+-------+
|321|Anubhaw|
|432| Ramesh|
|132| Manish|
|632|  Ankit|
|126|   Aman|
|732|   Ravi|
+---+-------+



In [8]:
customerDF.filter(customerDF['city']=='kota').show()

+---+----+---------+----+-------+
| id|name|    state|city|zipcode|
+---+----+---------+----+-------+
|732|Ravi|Rajasthan|kota| 246262|
+---+----+---------+----+-------+



In [9]:
customerDF.groupBy('city').count().show()

+----------+-----+
|      city|count|
+----------+-----+
|     Patna|    1|
|Jamshedpur|    2|
|  Amritsar|    1|
|      kota|    1|
|      Pune|    1|
+----------+-----+



### Create a `temp VIEW`  for running SQL queries on the dataframe

In [10]:
customerDF.createOrReplaceTempView('customers')

- SQL can be run on DataFrames that are registered as temp views.

In [11]:
cCityCount = spark.sql("select city,count(*) from customers group by city")

In [12]:
type(cCityCount)

pyspark.sql.dataframe.DataFrame

In [13]:
cCityCount.show()

+----------+--------+
|      city|count(1)|
+----------+--------+
|     Patna|       1|
|Jamshedpur|       2|
|  Amritsar|       1|
|      kota|       1|
|      Pune|       1|
+----------+--------+



In [14]:
cCityCount.printSchema()

root
 |-- city: string (nullable = true)
 |-- count(1): long (nullable = false)



### Writing dataframe to a file (parquet ,json etc)

In [20]:
# cCityCount.coalesce(1).write.parquet("CityCount_output.parquet")

In [16]:
# cCityCount.coalesce(1).write.save("count_output.parquet",format="parquet")

In [17]:
# cCityCount.coalesce(1).write.save("count_output3.parquet")

In [19]:
cCityCount.coalesce(1).write.json("count_output.json")

Py4JJavaError: An error occurred while calling o66.json.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:

### Dataframe from JSON

In [30]:
productDF=spark.read.load("sample.json",format="json")

#### Alternatively the following syntax is also allowed
` productDF = spark.read.json("sample.json") `
#### we can now run set of the DataFrame operations

In [31]:
productDF.printSchema()

root
 |-- color: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- size: string (nullable = true)



In [32]:
productDF.show()

+------+-------+-----+
| color|  fruit| size|
+------+-------+-----+
|   Red|  Apple|small|
|yellow|  Mango|small|
|yellow| Banana|Large|
| green| Grapes|small|
| brown|Coconut|Large|
+------+-------+-----+



In [33]:
productDF.select("color").show()

+------+
| color|
+------+
|   Red|
|yellow|
|yellow|
| green|
| brown|
+------+



In [34]:
productDF.select(['fruit','size','color']).show()

+-------+-----+------+
|  fruit| size| color|
+-------+-----+------+
|  Apple|small|   Red|
|  Mango|small|yellow|
| Banana|Large|yellow|
| Grapes|small| green|
|Coconut|Large| brown|
+-------+-----+------+



In [36]:
productDF.select(productDF['color'],productDF['fruit']).show()

+------+-------+
| color|  fruit|
+------+-------+
|   Red|  Apple|
|yellow|  Mango|
|yellow| Banana|
| green| Grapes|
| brown|Coconut|
+------+-------+



In [40]:
productDF.filter(productDF['size']=='small').show()

+------+------+-----+
| color| fruit| size|
+------+------+-----+
|   Red| Apple|small|
|yellow| Mango|small|
| green|Grapes|small|
+------+------+-----+



In [41]:
productDF.groupBy('size').count().show()

+-----+-----+
| size|count|
+-----+-----+
|small|    3|
|Large|    2|
+-----+-----+



#### Create `TEMP VIEW`  for running SQL queries

In [42]:
productDF.createOrReplaceTempView('products')

In [44]:
prod=spark.sql('select * from products order by size ')

In [46]:
type(prod)

pyspark.sql.dataframe.DataFrame

In [45]:
prod.show()

+------+-------+-----+
| color|  fruit| size|
+------+-------+-----+
|yellow| Banana|Large|
| brown|Coconut|Large|
|   Red|  Apple|small|
|yellow|  Mango|small|
| green| Grapes|small|
+------+-------+-----+



In [47]:
prod.printSchema()

root
 |-- color: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- size: string (nullable = true)



In [53]:
# prod.coalesce(1).write.save('product.parquet')

In [55]:
# prod.coalesce(1).write.json("product1.json")